In [1]:
import pandas as pd

In [2]:
from py2neo import Graph

In [4]:
graph = Graph(host='10.237.27.115')

In [5]:
locatedInDf = pd.read_csv('./finalLocatedInMatchedEntities.csv')

In [6]:
locatedInDf.shape

(186, 5)

In [7]:
locatedInDf.head()

,Unnamed: 0,org,loc,resolvedWith,sentences
0,0,ABG Container Handling Private Limited,['Mumbai'],['ABG CONTAINER HANDLING PRIVATE LIMITED'],['<ORG>Pacific International Lines</ORG> (<ORG...
1,1,ACE Designers Limited,['Bangalore'],['ACE DESIGNERS LIMITED'],['<ORG><MISC>SALEMDhirajlal Gandhi College of ...
2,2,Abhaya Narendra Lotha Foundation,['New Delhi'],['ABHAY NARENDRA LODHA FOUNDATION'],['Former <MISC>MP</MISC> <PER>Madhu Goud Yaskh...
3,3,Accenture,"['Bangalore', 'Bangalore']","['ACCENTURE INDIA PRIVATE LIMITED', 'ACCENTURE...","['<ORG>Accenture</ORG>, <LOC>Bangalore</LOC>, ..."
4,4,Aditya Birla,['Mumbai'],['ADITYA BIRLA SCIENCE AND TECHNOLOGY COMPANY ...,"[""R. Chidambaram (centre), Principal <MISC>Sci..."


In [8]:
locatedInDf['loc'] = locatedInDf['loc'].apply(eval)
locatedInDf['resolvedWith'] = locatedInDf['resolvedWith'].apply(eval)

In [9]:
#extracting rows in which companies are resolved with single organization in graph database
df = locatedInDf[locatedInDf['resolvedWith'].apply(lambda x: len(x) == 1)]
df.shape

(153, 5)

In [10]:
loc_query = 'match (c:city) where lower(c.name) = "{}" return c.uuid'

In [11]:
uuids = {}
uuids['Bangalore'] = 154538
uuids['Bengaluru'] = 154538
uuids['New Delhi'] = 154438
uuids['Kanagala'] = 154337 #district is guntur
uuids['Cochin'] = 154570 #district is ernakulam
uuids['Kochi'] = 154570 #district is ernakulam
uuids['Jamshedpur'] = 154519 #district is east singhbhum in jharkhand
for i in range(df.shape[0]):
    if df.iloc[i]['loc'][0] in uuids:
        continue
    res = graph.data(loc_query.format(df.iloc[i]['loc'][0].lower()))
    if len(res) != 0:
        uuids[df.iloc[i]['loc'][0]] = res[0]['c.uuid']
    else:
        #print(res, loc_query.format(df.iloc[i]['loc'][0]))
        print(df.iloc[i]['loc'][0])

London


In [12]:
comp_uuid = []
comp_query = 'match (c:company) where c.name = "{}" return c.uuid as uuid'
for i in range(df.shape[0]):
    try:
        comp_uuid.append(graph.data(comp_query.format(df.iloc[i]['resolvedWith'][0]))[0]['uuid'])
    except:
        print(df.iloc[i]['resolvedWith'][0])

In [13]:
maxRelIdQuery = 'match ()-[r]-() return max(r.relid) as maxRelId'
maxRelId = graph.data(maxRelIdQuery)[0]['maxRelId']
insertlocatedinRelation = 'match (x{{uuid:{0}}}),(y{{uuid:{2} }}) merge (x)-[r:locatedin {{bidirectional:false, reltype:"explicit", source:"news", relid:{1}}}]-> (y) return r.relid as id'
print (insertlocatedinRelation)

match (x{{uuid:{0}}}),(y{{uuid:{2} }}) merge (x)-[r:locatedin {{bidirectional:false, reltype:"explicit", source:"news", relid:{1}}}]-> (y) return r.relid as id


In [14]:
check_relation = 'match path=(c)-[:locatedin]-> (l) where c.uuid={} and l.uuid ={} return count(path) as c'
relId = maxRelId + 1
print('Starting Relation Id ', relId)
for i in range(df.shape[0]):
    try:
        if graph.data(check_relation.format(comp_uuid[i], uuids[df.iloc[i]['loc'][0]]))[0]['c'] != 0:
            print(df.iloc[i]['resolvedWith'][0], df.iloc[i]['loc'][0])
        else:
            res = graph.data(insertlocatedinRelation.format(comp_uuid[i], relId, uuids[df.iloc[i]['loc'][0]]))
            if len(res) == 0:
                print('Link not inserted')
            else:
                relId += 1
    except KeyError:
        print(df.iloc[i]['loc'][0])
print('End RelId', relId)

Starting Relation Id  617965
AVESTHAGEN LIMITED Bangalore
BHARAT PETROLEUM CORPORATION LIMITED Mumbai
HINDUSTAN AERONAUTICS LIMITED Bangalore
IBM INDIA PRIVATE LIMITED Bangalore
INDIAN OIL CORPN. LIMITED Bangalore
INFOSYS LIMITED Bangalore
INFOSYS LIMITED Bangalore
NATIONAL INSTRUMENTS LIMITED Bangalore
ROOTS INDUSTRIES INDIA LIMITED Coimbatore
TATA CONSULTANCY SERVICES LIMITED Bangalore
London
YOKOGAWA INDIA LIMITED Bangalore
End RelId 618106


In [15]:
relId - maxRelId

142